In [16]:
import pandas as pd
import requests
import time

In [17]:
def get_temp_vietnam(lat, lon, city_name):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date=2019-01-01&end_date=2025-10-31&daily=temperature_2m_mean,precipitation_sum&timezone=Asia%2FBangkok"
    response = requests.get(url).json()
    df = pd.DataFrame(response['daily'])
    df['time'] = pd.to_datetime(df['time'])
    # Chuyển đổi sang dữ liệu tháng
    df_monthly = df.set_index('time').resample('ME').agg({
        'temperature_2m_mean': 'mean',
        'precipitation_sum': 'sum'
    }).reset_index()
    df_monthly.columns = ['date', f'{city_name}_temp', f'{city_name}_precip']
    return df_monthly

In [18]:
def get_precipitation_data(lat, lon, region_name):
    """
    Lấy dữ liệu lượng mưa hàng ngày, sau đó resample sang tháng.
    """
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2019-01-01",
        "end_date": "2025-10-31",
        "daily": ["precipitation_sum"],
        "timezone": "Asia/Bangkok"
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['daily'])
        df['time'] = pd.to_datetime(df['time'])
        
        # Resample sang dữ liệu tháng (ME - Month End)
        # Lượng mưa dùng sum
        df_monthly = (
            df.set_index('time')
              .resample('ME')
              .agg({'precipitation_sum': 'sum'})
              .reset_index()
        )
        
        df_monthly.columns = ['date', f'precip_{region_name}']
        return df_monthly
    else:
        print(f"Lỗi khi lấy dữ liệu cho {region_name}")
        return None

In [19]:
def get_humidity_data(lat, lon, region_name):
    """
    Lấy dữ liệu độ ẩm trung bình hàng ngày,
    sau đó resample sang tháng.
    """
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2019-01-01",
        "end_date": "2025-10-31",
        "daily": ["relative_humidity_2m_mean"],
        "timezone": "Asia/Bangkok"
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["daily"])
        df["time"] = pd.to_datetime(df["time"])

        # Resample sang dữ liệu tháng (ME - Month End)
        # Độ ẩm dùng mean
        df_monthly = (
            df.set_index("time")
              .resample("ME")
              .agg({"relative_humidity_2m_mean": "mean"})
              .reset_index()
        )

        df_monthly.columns = ["date", f"humidity_{region_name}"]
        return df_monthly
    else:
        print(f"Lỗi khi lấy dữ liệu cho {region_name}")
        return None


In [27]:
def get_solar_radiation_data(lat, lon, region_name):
    """
    Lấy dữ liệu bức xạ mặt trời (shortwave radiation) hàng ngày,
    sau đó resample sang tháng.
    """
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2019-01-01",
        "end_date": "2025-10-31",
        "daily": ["shortwave_radiation_sum"],
        "timezone": "Asia/Bangkok"
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["daily"])
        df["time"] = pd.to_datetime(df["time"])

        # Resample sang dữ liệu tháng (ME - Month End)
        # Bức xạ dùng sum (tổng năng lượng trong tháng)
        df_monthly = (
            df.set_index("time")
              .resample("ME")
              .agg({"shortwave_radiation_sum": "sum"})
              .reset_index()
        )

        df_monthly.columns = ["date", f"solar_{region_name}"]
        return df_monthly
    else:
        print(f"Lỗi khi lấy dữ liệu cho {region_name}")
        return None


In [20]:
# Tọa độ 3 miền
hanoi_lat, hanoi_lon = 21.02, 105.85
danang_lat, danang_lon = 16.04, 108.20
hcm_lat, hcm_lon = 10.82, 106.62

In [21]:
# Lấy dữ liệu nhiệt độ
hanoi = get_temp_vietnam(hanoi_lat, hanoi_lon, "HN")
danang = get_temp_vietnam(danang_lat, danang_lon, "DN")
hcm = get_temp_vietnam(hcm_lat, hcm_lon, "HCM")

In [22]:
# Gộp lại thành một bảng duy nhất
weather_df = hanoi.merge(danang, on='date').merge(hcm, on='date')
weather_df.to_csv("weather_vietnam_tft.csv", index=False)

In [23]:
# Lấy dữ liệu lượng mưa cho 3 miền
precip_hanoi = get_precipitation_data(hanoi_lat, hanoi_lon, "HN")
precip_danang = get_precipitation_data(danang_lat, danang_lon, "DN")
precip_hcm = get_precipitation_data(hcm_lat, hcm_lon, "HCM")

In [24]:
# Gộp lại thành một bảng duy nhất
precip_df = precip_hanoi.merge(precip_danang, on='date').merge(precip_hcm, on='date')
precip_df.to_csv("precipitation_vietnam_tft.csv", index=False)

In [25]:
# Lấy dữ liệu độ ẩm cho 3 miền
humidity_hanoi = get_humidity_data(hanoi_lat, hanoi_lon, "HN")
humidity_danang = get_humidity_data(danang_lat, danang_lon, "DN")
humidity_hcm = get_humidity_data(hcm_lat, hcm_lon, "HCM")

In [26]:
# Gộp lại thành một bảng duy nhất
humidity_df = humidity_hanoi.merge(humidity_danang, on='date').merge(humidity_hcm, on='date')
humidity_df.to_csv("humidity_vietnam_tft.csv", index=False)

In [28]:
# Lấy dữ liệu bức xạ mặt trời cho 3 miền
solar_hanoi = get_solar_radiation_data(hanoi_lat, hanoi_lon, "HN")
solar_danang = get_solar_radiation_data(danang_lat, danang_lon, "DN")
solar_hcm = get_solar_radiation_data(hcm_lat, hcm_lon, "HCM")

In [29]:
# Gộp lại thành một bảng duy nhất
solar_df = solar_hanoi.merge(solar_danang, on='date').merge(solar_hcm, on='date')
solar_df.to_csv("solar_radiation_vietnam_tft.csv", index=False)